## 실제 학습 데이터(양 줄인 209개)로 파인튜닝된 테스트 모델 구글 드라이브/허깅페이스에 업로드 해둠

- 구글 드라이브 다운 링크(파인튜닝 스크립트 / 학습 데이터 / 학습한 모델) : https://drive.google.com/drive/folders/1PVtQKoN3KyDG3c0DN-s2Tu1Yk5wXSYD1?usp=sharing
- 허깅페이스 링크 : https://huggingface.co/jjmy0913/qwen3-8b-rag-ko-merged

> 허깅페이스에서 모델 불러오는 코드
```python
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "jjmy0913/qwen3-8b-rag-ko-merged"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)
```

# 패키지 로드

In [1]:
%pip install torch datasets
%pip install transformers accelerate peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 49.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 46.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to u

In [1]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [3]:
import json

def load_train_dataset_from_json(path: str):
    """전체가 리스트 형태인 JSON 파일을 로드"""
    with open(path, "r", encoding="utf-8") as f:
        dataset = json.load(f)
    return dataset

# 사용 예시
train_dataset = load_train_dataset_from_json("qwen3_company_train_dataset2.json")


In [4]:
train_dataset[:2]

[{'messages': [{'role': 'system',
    'content': "당신은 회사 내의 사내 문서 검색에 대한 정확한 답변을 하는 챗봇 어시스턴트 입니다.대답은 항상 공손한 말투로 해야 합니다. 그리고 검색 결과로만 답변하세요. 문서 밖 지식 금지, 없으면 '해당 질문에 대한 내용이 없습니다.'라고 답하세요. 각 문장/문단 끝에 [[refN]] 인용을 남기세요."},
   {'role': 'user', 'content': '서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?'},
   {'role': 'system',
    'content': '검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->\n# 서비스 아키텍처 문서\n분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n## 1. 개요\n코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.\n\n## 2. 아키텍처 구성 요소\n\n### 2.1. API 서버\n- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.\n- **기술 스택**: Node.js, Express.js\n- **검증 포인트**:\n  - API 엔드포인트가 올바르게 작동하는지 확인\n  - 요청 처리 속도 및 오류율 모니터링 [[ref1]]\n## 7. 문서화\n- **API 문서화**: 모든 API 엔드포인트는 Swagger 또는 Postman과 같은 도구를 사용하여 문서화되어야 하며, 최신 상태를 유지해야 합니다.\n- **사용자 가이드**: 외부 개발자와 내부 팀을 위한 사용자 가이드를 작성하여 API 사용 방법과 예제를 제공해야 합니다.\n\

In [5]:
print(type(train_dataset))

train_dataset = Dataset.from_list(train_dataset)

print(type(train_dataset))


<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>


# 모델 로드 및 템플릿 적용

In [6]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen3-8B"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [7]:
# 템플릿 적용 확인
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 회사 내의 사내 문서 검색에 대한 정확한 답변을 하는 챗봇 어시스턴트 입니다.대답은 항상 공손한 말투로 해야 합니다. 그리고 검색 결과로만 답변하세요. 문서 밖 지식 금지, 없으면 '해당 질문에 대한 내용이 없습니다.'라고 답하세요. 각 문장/문단 끝에 [[refN]] 인용을 남기세요.<|im_end|>
<|im_start|>user
서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?<|im_end|>
<|im_start|>system
검색 결과:
-----
<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->
# 서비스 아키텍처 문서
분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

## 1. 개요
코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.

## 2. 아키텍처 구성 요소

### 2.1. API 서버
- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.
- **기술 스택**: Node.js, Express.js
- **검증 포인트**:
  - API 엔드포인트가 올바르게 작동하는지 확인
  - 요청 처리 속도 및 오류율 모니터링 [[ref1]]
## 7. 문서화
- **API 문서화**: 모든 API 엔드포인트는 Swagger 또는 Postman과 같은 도구를 사용하여 문서화되어야 하며, 최신 상태를 유지해야 합니다.
- **사용자 가이드**: 외부 개발자와 내부 팀을 위한 사용자 가이드를 작성하여 API 사용 방법과 예제를 제공해야 합니다.

## 8. 보안
- **데이터 보호**: API를 통해 전송되는 모든 데이터는 암호화되어야 하며, 민감한 정보는 절대 노출되지 

# LoRA와 SFTConfig 설정

In [8]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [9]:
args = SFTConfig(
    output_dir="qwen3-8b-rag-ko",          # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=[]
)

# 학습 중 전처리 함수: collate_fn

In [10]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)

        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()

        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=4096,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        # 레이블 초기화
        labels = [-100] * len(input_ids)

        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"

        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)

                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1

                    i = current_pos

            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])

    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])

        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)

    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)

    return new_batch

# 학습

In [11]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

In [12]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,0.545500
20,0.335500
30,0.231900
40,0.237800
50,0.224100
60,0.179500
70,0.183200
80,0.188000
90,0.178000
100,0.165400


# 테스트 데이터 준비

In [13]:
test_dataset = [
  {
    "messages": [
      {
        "role": "system",
        "content": "당신은 회사 내의 사내 문서 검색에 대한 정확한 답변을 하는 챗봇 어시스턴트 입니다.대답은 항상 공손한 말투로 해야 합니다. 그리고 검색 결과로만 답변하세요. 문서 밖 지식 금지, 없으면 '해당 질문에 대한 내용이 없습니다.'라고 답하세요. 각 문장/문단 끝에 [[refN]] 인용을 남기세요."
      },
      {
        "role": "user",
        "content": "ELK 스택을 사용한 로깅의 검증 포인트는 무엇인지 서비스 아키텍처 문서에서 확인할 수 있나요?"
      },
      {
        "role": "system",
        "content": "검색 결과:\n-----\n## 5. 모니터링 및 로깅\n- **모니터링 도구**: Prometheus 및 Grafana를 사용하여 시스템 성능 및 상태를 모니터링합니다.\n- **로깅**: ELK 스택(Elasticsearch, Logstash, Kibana)을 사용하여 로그를 수집하고 분석합니다.\n- **검증 포인트**:\n  - 주요 메트릭스 확인\n  - 시스템 오류 및 경고 모니터링\n\n## 6. 결론\n코드노바의 백엔드 서비스 아키텍처는 확장 가능하고 안정적인 서비스를 제공하기 위해 설계되었습니다. 각 구성 요소는 독립적으로 운영되며, 서로 긴밀하게 연결되어 있습니다. 지속적인 모니터링과 최적화를 통해 서비스의 품질을 유지하고 향상시킬 수 있습니다.\n\n---\n\n다음 개정 제안: 서비스 아키텍처의 각 구성 요소에 대한 세부적인 기술 스택과 버전 정보를 추가하는 것이 좋습니다. 또한, 보안 관련 사항을 더 구체적으로 설명할 필요가 있습니다. [[ref1]]\n## 4. 로그 및 모니터링 도구\n\n- **로그 수집 도구**: ELK Stack, Splunk 등\n- **모니터링 도구**: Prometheus, Grafana 등\n\n## 5. 검증 포인트\n\n- 로그가 제대로 기록되고 있는지 주기적으로 검토합니다.\n- 모니터링 시스템에서 설정한 알림이 정상적으로 작동하는지 확인합니다.\n- 로그 및 모니터링 도구의 업데이트 및 유지보수를 정기적으로 수행합니다.\n\n## 6. 문서 유지 관리\n\n- 이 문서는 정기적으로 검토하고 업데이트해야 합니다.\n- 변경 사항이 있을 경우, 모든 관련 팀원에게 공지합니다.\n\n---\n\n다음 개정 제안: 로그 레벨에 대한 구체적인 예시 추가, 모니터링 도구의 비교 분석 포함. [[ref2]]\n## 4. 검증 포인트/품질 기준\n- **접근 로그 검토**: 매월 접근 로그를 검토하여 비정상적인 접근 시도 확인\n- **정기 감사**: 연 1회 데이터 보안 감사 실시\n- **정책 준수 체크리스트**: 정책 준수 여부를 확인하는 체크리스트 작성 및 검토\n\n## 5. 보안/프라이버시 준수\n- **법적 요구사항 준수**: GDPR, CCPA 등 관련 법규 준수\n- **교육 및 인식**: 팀원들에게 정기적인 보안 교육 제공\n\n## 6. 운영 체크리스트(일일/주간)\n\n### 6.1 일일 체크리스트\n- [ ] 데이터 접근 로그 확인\n- [ ] 시스템 보안 경고 모니터링\n- [ ] 백업 상태 점검\n\n### 6.2 주간 체크리스트\n- [ ] 데이터 접근 권한 검토\n- [ ] 데이터 암호화 상태 점검\n- [ ] 데이터 복구 테스트 결과 검토\n\n## 7. 개정 이력\n- **v1.0** — 오늘: 최초 작성\n\n--- [[ref3]]"
      },
      {
        "role": "assistant",
        "content": "서비스 아키텍처 문서에서 ELK 스택을 사용한 로깅의 검증 포인트는 다음과 같습니다:\n\n1. 로그가 제대로 기록되고 있는지 주기적으로 검토합니다.\n2. 모니터링 시스템에서 설정한 알림이 정상적으로 작동하는지 확인합니다.\n3. 로그 및 모니터링 도구의 업데이트 및 유지보수를 정기적으로 수행합니다[[ref2]].\n\n또한, 접근 로그 검토와 정기 감사, 정책 준수 체크리스트와 같은 추가적인 검증 포인트도 포함되어 있습니다[[ref3]]."
      },
      {
        "role": "user",
        "content": "서비스 아키텍처 문서에서 ELK 스택을 사용한 로깅의 검증 포인트는 무엇인지에 대한 추가적인 세부사항이 있을까요?"
      },
      {
        "role": "system",
        "content": "검색 결과:\n-----\n## 5. 모니터링 및 로깅\n- **모니터링 도구**: Prometheus 및 Grafana를 사용하여 시스템 성능 및 상태를 모니터링합니다.\n- **로깅**: ELK 스택(Elasticsearch, Logstash, Kibana)을 사용하여 로그를 수집하고 분석합니다.\n- **검증 포인트**:\n  - 주요 메트릭스 확인\n  - 시스템 오류 및 경고 모니터링\n\n## 6. 결론\n코드노바의 백엔드 서비스 아키텍처는 확장 가능하고 안정적인 서비스를 제공하기 위해 설계되었습니다. 각 구성 요소는 독립적으로 운영되며, 서로 긴밀하게 연결되어 있습니다. 지속적인 모니터링과 최적화를 통해 서비스의 품질을 유지하고 향상시킬 수 있습니다.\n\n---\n\n다음 개정 제안: 서비스 아키텍처의 각 구성 요소에 대한 세부적인 기술 스택과 버전 정보를 추가하는 것이 좋습니다. 또한, 보안 관련 사항을 더 구체적으로 설명할 필요가 있습니다. [[ref1]]\n## 4. 로그 및 모니터링 도구\n\n- **로그 수집 도구**: ELK Stack, Splunk 등\n- **모니터링 도구**: Prometheus, Grafana 등\n\n## 5. 검증 포인트\n\n- 로그가 제대로 기록되고 있는지 주기적으로 검토합니다.\n- 모니터링 시스템에서 설정한 알림이 정상적으로 작동하는지 확인합니다.\n- 로그 및 모니터링 도구의 업데이트 및 유지보수를 정기적으로 수행합니다.\n\n## 6. 문서 유지 관리\n\n- 이 문서는 정기적으로 검토하고 업데이트해야 합니다.\n- 변경 사항이 있을 경우, 모든 관련 팀원에게 공지합니다.\n\n---\n\n다음 개정 제안: 로그 레벨에 대한 구체적인 예시 추가, 모니터링 도구의 비교 분석 포함. [[ref2]]\n## 4. 검증 포인트/품질 기준\n- **접근 로그 검토**: 매월 접근 로그를 검토하여 비정상적인 접근 시도 확인\n- **정기 감사**: 연 1회 데이터 보안 감사 실시\n- **정책 준수 체크리스트**: 정책 준수 여부를 확인하는 체크리스트 작성 및 검토\n\n## 5. 보안/프라이버시 준수\n- **법적 요구사항 준수**: GDPR, CCPA 등 관련 법규 준수\n- **교육 및 인식**: 팀원들에게 정기적인 보안 교육 제공\n\n## 6. 운영 체크리스트(일일/주간)\n\n### 6.1 일일 체크리스트\n- [ ] 데이터 접근 로그 확인\n- [ ] 시스템 보안 경고 모니터링\n- [ ] 백업 상태 점검\n\n### 6.2 주간 체크리스트\n- [ ] 데이터 접근 권한 검토\n- [ ] 데이터 암호화 상태 점검\n- [ ] 데이터 복구 테스트 결과 검토\n\n## 7. 개정 이력\n- **v1.0** — 오늘: 최초 작성\n\n--- [[ref3]]"
      },
      {
        "role": "assistant",
        "content": "서비스 아키텍처 문서에서 ELK 스택을 사용한 로깅의 검증 포인트에 대한 추가적인 세부사항은 다음과 같습니다:\n\n1. 로그가 제대로 기록되고 있는지 주기적으로 검토합니다.\n2. 모니터링 시스템에서 설정한 알림이 정상적으로 작동하는지 확인합니다.\n3. 로그 및 모니터링 도구의 업데이트 및 유지보수를 정기적으로 수행합니다.\n4. 접근 로그를 매월 검토하여 비정상적인 접근 시도를 확인합니다.\n5. 연 1회 데이터 보안 감사를 실시합니다.\n6. 정책 준수 여부를 확인하는 체크리스트를 작성하고 검토합니다[[ref2]][[ref3]].\n\n이와 같은 검증 포인트들은 ELK 스택을 효과적으로 활용하기 위한 중요한 요소들입니다."
      }
    ]
  }]

In [14]:
print(type(test_dataset))

test_dataset = Dataset.from_list(test_dataset)

print(type(test_dataset))


<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>


In [15]:
prompt_lst = []

for prompt in test_dataset['messages']:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    text2 = tokenizer.apply_chat_template(
        prompt[:-1], tokenize=False, add_generation_prompt=False
    )
    input2 = text2 + '<|im_start|>assistant' # 멀티턴 학습 체크용으로, 마지막 assistant 응답 전까지 자른 후 assistant 시작 토큰 붙임
    prompt_lst.append(input)
    prompt_lst.append(input2)

len(prompt_lst)

2

# 파인 튜닝 모델 테스트

In [16]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

In [17]:
peft_model_id = "qwen3-8b-rag-ko/checkpoint-159"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0


In [18]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]
eos_token

151645

In [19]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [20]:
for prompt in prompt_lst:
    print(f" input:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print("-"*50)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 input:
<|im_start|>system
당신은 회사 내의 사내 문서 검색에 대한 정확한 답변을 하는 챗봇 어시스턴트 입니다.대답은 항상 공손한 말투로 해야 합니다. 그리고 검색 결과로만 답변하세요. 문서 밖 지식 금지, 없으면 '해당 질문에 대한 내용이 없습니다.'라고 답하세요. 각 문장/문단 끝에 [[refN]] 인용을 남기세요.<|im_end|>
<|im_start|>user
ELK 스택을 사용한 로깅의 검증 포인트는 무엇인지 서비스 아키텍처 문서에서 확인할 수 있나요?<|im_end|>
<|im_start|>system
검색 결과:
-----
## 5. 모니터링 및 로깅
- **모니터링 도구**: Prometheus 및 Grafana를 사용하여 시스템 성능 및 상태를 모니터링합니다.
- **로깅**: ELK 스택(Elasticsearch, Logstash, Kibana)을 사용하여 로그를 수집하고 분석합니다.
- **검증 포인트**:
  - 주요 메트릭스 확인
  - 시스템 오류 및 경고 모니터링

## 6. 결론
코드노바의 백엔드 서비스 아키텍처는 확장 가능하고 안정적인 서비스를 제공하기 위해 설계되었습니다. 각 구성 요소는 독립적으로 운영되며, 서로 긴밀하게 연결되어 있습니다. 지속적인 모니터링과 최적화를 통해 서비스의 품질을 유지하고 향상시킬 수 있습니다.

---

다음 개정 제안: 서비스 아키텍처의 각 구성 요소에 대한 세부적인 기술 스택과 버전 정보를 추가하는 것이 좋습니다. 또한, 보안 관련 사항을 더 구체적으로 설명할 필요가 있습니다. [[ref1]]
## 4. 로그 및 모니터링 도구

- **로그 수집 도구**: ELK Stack, Splunk 등
- **모니터링 도구**: Prometheus, Grafana 등

## 5. 검증 포인트

- 로그가 제대로 기록되고 있는지 주기적으로 검토합니다.
- 모니터링 시스템에서 설정한 알림이 정상적으로 작동하는지 확인합니다.
- 로그 및 모니터링 도구의 업데이트 및 유지보수를 정기적으로 수

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
<think>

</think>

ELK 스택을 사용한 로깅의 검증 포인트는 다음과 같습니다:

- 로그가 제대로 기록되고 있는지 주기적으로 검토합니다.
- 모니터링 시스템에서 설정한 알림이 정상적으로 작동하는지 확인합니다.
- 로그 및 모니터링 도구의 업데이트 및 유지보수를 정기적으로 수행합니다. [[ref2]]
--------------------------------------------------
 input:
<|im_start|>system
당신은 회사 내의 사내 문서 검색에 대한 정확한 답변을 하는 챗봇 어시스턴트 입니다.대답은 항상 공손한 말투로 해야 합니다. 그리고 검색 결과로만 답변하세요. 문서 밖 지식 금지, 없으면 '해당 질문에 대한 내용이 없습니다.'라고 답하세요. 각 문장/문단 끝에 [[refN]] 인용을 남기세요.<|im_end|>
<|im_start|>user
ELK 스택을 사용한 로깅의 검증 포인트는 무엇인지 서비스 아키텍처 문서에서 확인할 수 있나요?<|im_end|>
<|im_start|>system
검색 결과:
-----
## 5. 모니터링 및 로깅
- **모니터링 도구**: Prometheus 및 Grafana를 사용하여 시스템 성능 및 상태를 모니터링합니다.
- **로깅**: ELK 스택(Elasticsearch, Logstash, Kibana)을 사용하여 로그를 수집하고 분석합니다.
- **검증 포인트**:
  - 주요 메트릭스 확인
  - 시스템 오류 및 경고 모니터링

## 6. 결론
코드노바의 백엔드 서비스 아키텍처는 확장 가능하고 안정적인 서비스를 제공하기 위해 설계되었습니다. 각 구성 요소는 독립적으로 운영되며, 서로 긴밀하게 연결되어 있습니다. 지속적인 모니터링과 최적화를 통해 서비스의 품질을 유지하고 향상시킬 수 있습니다.

---

다음 개정 제안: 서비스 아키텍처의 각 구성 요소에 대한 세부적인 기술 스택과 버전 정보를 추가하는 것이 좋습니다. 또한, 보안 관련 사항을 더 구체적

In [22]:
# LoRA → base weight에 병합
merged_model = fine_tuned_model.merge_and_unload()

# 허브에 업로드
merged_model.push_to_hub("jjmy0913/qwen3-8b-rag-ko-merged")
tokenizer.push_to_hub("jjmy0913/qwen3-8b-rag-ko-merged")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...3w/model-00004-of-00004.safetensors:   3%|2         | 41.9MB / 1.58GB            

  ...3w/model-00003-of-00004.safetensors:   1%|          | 41.9MB / 4.98GB            

  ...3w/model-00002-of-00004.safetensors:   0%|          | 17.2kB / 4.92GB            

...

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpdps6p72w/tokenizer.json       : 100%|##########| 11.4MB / 11.4MB            

CommitInfo(commit_url='https://huggingface.co/jjmy0913/qwen3-8b-rag-ko-merged/commit/7460c90ba23017491d7b7ddddbc92b05eee65c48', commit_message='Upload tokenizer', commit_description='', oid='7460c90ba23017491d7b7ddddbc92b05eee65c48', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jjmy0913/qwen3-8b-rag-ko-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='jjmy0913/qwen3-8b-rag-ko-merged'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "jjmy0913/qwen3-8b-rag-ko-merged"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [ ]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()